<table align="left">
  <td>
    <a href="https://is.gd/srnfvZ" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

## Ejemplo 2. Preprocesamiento aplicando Clustering

Vamos a abordar el conjunto de datos _digits_, que es un conjunto de datos sencillo similar al MNIST que contiene 1.797 imágenes de 8×8 en escala de grises que representan los dígitos del 0 al 9.

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


from sklearn.cluster import KMeans
from sklearn.datasets import load_digits

In [2]:
X_digits, y_digits = load_digits(return_X_y=True)

### Train - Test split (80-20)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, random_state=42)

Ahora vamos a ajustar un modelo de regresión logística y a evaluarlo en el conjunto de pruebas:

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=5000, multi_class='ovr', random_state=42)

In [6]:
log_reg_score = log_reg.score(X_test, y_test)
log_reg_score

0.9688888888888889

Bien, esa es nuestra línea de base: 96,89% de precisión. Veamos si podemos hacerlo mejor utilizando K-Means como paso previo al procesamiento. 

Crearemos un pipeline que primero agrupará el conjunto de entrenamiento en **50 clusters** y reemplazará las imágenes con sus distancias a los **50 clusters**, y luego se aplicará un modelo de regresión logística.


> El pipeline es sólo una noción abstracta, no es un algoritmo de ML existente. A menudo, en las tareas de ML es necesario realizar una secuencia de diferentes transformaciones (encontrar un conjunto de características, generar nuevas características, seleccionar sólo algunas buenas características) del conjunto de datos en bruto antes de aplicar el estimador final.



In [7]:
from sklearn.pipeline import Pipeline

In [15]:
pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=200, random_state=42)),
    ("log_reg", LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)),
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('kmeans', KMeans(n_clusters=200, random_state=42)),
                ('log_reg',
                 LogisticRegression(max_iter=5000, multi_class='ovr',
                                    random_state=42))])

In [16]:
pipeline_score = pipeline.score(X_test, y_test)
pipeline_score

0.9822222222222222

¿Cuánto ha bajado la tasa de error?

In [17]:
1 - (1 - pipeline_score) / (1 - log_reg_score)

0.42857142857142805

Hemos reducido la tasa de error en más de un 28.6%. Pero hemos elegido el número de clusters k de forma totalmente arbitraria, seguramente podemos hacerlo mejor. 

Dado que K-Means es sólo un paso de preprocesamiento en un proceso de clasificación, encontrar un buen valor para k es mucho más sencillo que antes: no hay necesidad de realizar un análisis de silueta o minimizar la inercia, el mejor valor de k es simplemente el que resulta en el mejor rendimiento de clasificación.

In [ ]:
from sklearn.model_selection import GridSearchCV

**Atención**: el siguiente bloque puede tardar cerca de 20 minutos en ejecutarse, o más dependiendo de su hardware.

In [ ]:
param_grid = dict(kmeans__n_clusters=range(100, 200))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits
[CV] END ...............................kmeans__n_clusters=2; total time=   0.3s
[CV] END ...............................kmeans__n_clusters=2; total time=   0.4s
[CV] END ...............................kmeans__n_clusters=2; total time=   0.3s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.5s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.3s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.3s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.3s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.4s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.4s
[CV] END ...............................kmeans__n_clusters=5; total time=   0.5s
[CV] END ...............................kmeans__n_clusters=5; total time=   0.5s
[CV] END ...............................kmeans_

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ..............................kmeans__n_clusters=55; total time=   7.6s
[CV] END ..............................kmeans__n_clusters=56; total time=   5.6s
[CV] END ..............................kmeans__n_clusters=56; total time=   5.1s
[CV] END ..............................kmeans__n_clusters=56; total time=   5.7s
[CV] END ..............................kmeans__n_clusters=57; total time=   5.7s
[CV] END ..............................kmeans__n_clusters=57; total time=   6.4s
[CV] END ..............................kmeans__n_clusters=57; total time=   5.6s
[CV] END ..............................kmeans__n_clusters=58; total time=   6.0s
[CV] END ..............................kmeans__n_clusters=58; total time=   6.6s
[CV] END ..............................kmeans__n_clusters=58; total time=   5.2s
[CV] END ..............................kmeans__n_clusters=59; total time=   6.1s
[CV] END ..............................kmeans__n_clusters=59; total time=   6.2s
[CV] END ...................

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans',
                                        KMeans(n_clusters=50, random_state=42)),
                                       ('log_reg',
                                        LogisticRegression(max_iter=5000,
                                                           multi_class='ovr',
                                                           random_state=42))]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

Veamos cuál es el mejor número de Clusters:

In [ ]:
grid_clf.best_params_

{'kmeans__n_clusters': 95}

In [ ]:
grid_clf.score(X_test, y_test)

0.9777777777777777